# Projeto IA AV3 - Classificação de Energia

**Disciplina:** Inteligência Artificial Computacional

**Aluno:** Mateus Gomes Macário

**Professor:** Ms. Cynthia Moreira Maia

---

## 📌 Objetivo

Implementar e avaliar algoritmos de classificação **do zero** (sem scikit-learn ou pandas):
- K-Nearest Neighbors (KNN)
- Perceptron
- Multi-Layer Perceptron (MLP)
- Naive Bayes

## 🔗 Links

- **Repositório:** [github.com/mateusmacario/electrolyzer-simulator](https://github.com/mateusmacario/electrolyzer-simulator)
- **Dataset:** [OpenML - Appliances Energy](https://www.openml.org/d/46283)


## 1️⃣ Setup e Instalação

In [ ]:
# Clone do repositório
!git clone https://github.com/mateusmacario/electrolyzer-simulator.git
%cd electrolyzer-simulator/projeto_ia_av3

In [ ]:
# Instala dependências
!pip install -q numpy matplotlib

## 2️⃣ Importações

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Adiciona src ao path
sys.path.insert(0, 'src')

# Imports dos algoritmos
from algorithms.knn import KNNEuclidean, KNNManhattan
from algorithms.perceptron import MultiClassPerceptron
from algorithms.mlp import MLP
from algorithms.naive_bayes import UnivariateNaiveBayes, MultivariateNaiveBayes

# Imports dos utilitários
from utils.preprocessing import StandardScaler, binarize_target
from utils.cross_validation import cross_validate_stratified
from utils.visualization import plot_metrics_comparison, generate_markdown_table

print("✓ Importações concluídas!")

## 3️⃣ Download e Carregamento do Dataset

In [ ]:
# Opção 1: Criar dataset sintético para demonstração
!python src/create_demo_dataset.py

In [ ]:
# Carrega dataset
import csv

def load_dataset(filepath='data/raw/appliances_energy.csv'):
    data = []
    with open(filepath, 'r') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            try:
                data.append([float(val) for val in row])
            except:
                continue
    
    data = np.array(data)
    X = data[:, :-1]
    y = data[:, -1]
    
    return X, y

X, y = load_dataset()
print(f"Dataset carregado: {X.shape[0]} amostras, {X.shape[1]} features")

## 4️⃣ Pré-processamento

In [ ]:
# Binariza target
y_binary = binarize_target(y)
print(f"Classe 0: {np.sum(y_binary == 0)} amostras")
print(f"Classe 1: {np.sum(y_binary == 1)} amostras")

# Normaliza features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)
print("\n✓ Pré-processamento concluído!")

## 5️⃣ Definição dos Modelos

In [ ]:
models = {
    'KNN (Euclidiana)': KNNEuclidean(k=5),
    'KNN (Manhattan)': KNNManhattan(k=5),
    'Perceptron': MultiClassPerceptron(learning_rate=0.01, n_epochs=50),
    'MLP': MLP(input_size=X_normalized.shape[1], hidden_sizes=[32, 16],
               output_size=2, learning_rate=0.01, n_epochs=50),
    'Naive Bayes (Univariado)': UnivariateNaiveBayes(),
    'Naive Bayes (Multivariado)': MultivariateNaiveBayes()
}

print(f"Modelos definidos: {len(models)}")
for name in models.keys():
    print(f"  - {name}")

## 6️⃣ Validação Cruzada (K-Fold)

In [ ]:
results = {}
n_folds = 5

for model_name, model in models.items():
    print(f"\n{'=' * 60}")
    print(f"Avaliando: {model_name}")
    print('=' * 60)
    
    cv_results = cross_validate_stratified(
        model, X_normalized, y_binary,
        n_folds=n_folds,
        verbose=True
    )
    
    results[model_name] = cv_results
    
    print(f"\nResumo:")
    print(f"  Acurácia:  {cv_results['accuracy_mean']:.4f} ± {cv_results['accuracy_std']:.4f}")
    print(f"  F1-Score:  {cv_results['f1_score_mean']:.4f} ± {cv_results['f1_score_std']:.4f}")

print("\n✓ Validação cruzada concluída!")

## 7️⃣ Resultados e Visualizações

In [ ]:
# Gera tabela de resultados
md_table = generate_markdown_table(results)
print(md_table)

In [ ]:
# Visualiza comparação de métricas
plot_metrics_comparison(results)
plt.show()

In [ ]:
# Análise do melhor modelo
best_model = max(results.items(), key=lambda x: x[1]['accuracy_mean'])
print(f"\n🏆 MELHOR MODELO: {best_model[0]}")
print(f"   Acurácia: {best_model[1]['accuracy_mean']:.4f} ± {best_model[1]['accuracy_std']:.4f}")
print(f"   F1-Score: {best_model[1]['f1_score_mean']:.4f} ± {best_model[1]['f1_score_std']:.4f}")

## 8️⃣ Análise de Trade-off

In [ ]:
print("\nANÁLISE DE TRADE-OFF (Desempenho vs Eficiência):")
print("=" * 70)

efficiency_scores = {}
for model_name, result in results.items():
    acc = result['accuracy_mean']
    time_total = result['train_time_mean'] + result['test_time_mean']
    efficiency = acc / (time_total + 0.01)
    efficiency_scores[model_name] = efficiency
    
    print(f"{model_name}")
    print(f"  Acurácia: {acc:.4f} | Tempo: {time_total:.2f}s | Score: {efficiency:.4f}")

best_efficiency = max(efficiency_scores.items(), key=lambda x: x[1])
print(f"\n✨ Melhor Trade-off: {best_efficiency[0]} (Score: {best_efficiency[1]:.4f})")

## 9️⃣ Exportação dos Resultados

In [ ]:
# Salva resultados em arquivo
import json

# Prepara resultados para serialização
results_serializable = {}
for model_name, result in results.items():
    results_serializable[model_name] = {
        'accuracy_mean': float(result['accuracy_mean']),
        'accuracy_std': float(result['accuracy_std']),
        'f1_score_mean': float(result['f1_score_mean']),
        'f1_score_std': float(result['f1_score_std']),
        'train_time_mean': float(result['train_time_mean']),
        'test_time_mean': float(result['test_time_mean'])
    }

# Salva como JSON
with open('results_colab.json', 'w') as f:
    json.dump(results_serializable, f, indent=2)

print("✓ Resultados salvos em results_colab.json")
print("✓ Você pode baixar este arquivo para integrar com sua aplicação web!")

## 🎉 Conclusão

Todos os algoritmos foram implementados **100% manualmente** sem uso de bibliotecas de ML!

### Principais Resultados:
- ✅ Todos os requisitos do projeto atendidos
- ✅ Código limpo e bem documentado
- ✅ Validação cruzada implementada do zero
- ✅ Métricas calculadas manualmente

### Próximos Passos:
1. Baixar dataset real do OpenML
2. Testar com dataset completo (19,735 amostras)
3. Ajustar hiperparâmetros
4. Integrar com aplicação web

---

**Desenvolvido por:** Mateus Gomes Macário

**Disciplina:** Inteligência Artificial Computacional - UNIFOR
